In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from mlxtend.classifier import StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, balanced_accuracy_score
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report, precision_recall_curve
from sklearn.model_selection import train_test_split
from imblearn.metrics import geometric_mean_score, sensitivity_score, specificity_score
from xgboost import XGBClassifier
from shaphypetune import BoostRFE
from sklearn.base import BaseEstimator
from sklearn.utils.metaestimators import if_delegate_has_method

import torch
import re, pickle, random, os
import warnings
warnings.filterwarnings('ignore')

from collections import Counter

In [2]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

RANDOM_STATE = 42
seed_everything(seed=RANDOM_STATE)

In [3]:
col_names = ['name','hobby','age','edu','marital','class']
df = pd.read_csv('hayes-roth.csv', names = col_names)
df.shape

(132, 6)

In [4]:
df.head()

,name,hobby,age,edu,marital,class
0,92,2,1,1,2,1
1,10,2,1,3,2,2
2,83,3,1,4,1,3
3,61,2,4,2,2,3
4,107,1,1,3,4,3


In [5]:
df['class'].value_counts()

1    51
2    51
3    30
Name: class, dtype: int64

In [6]:
df.isnull().sum()

name       0
hobby      0
age        0
edu        0
marital    0
class      0
dtype: int64

In [7]:
df.describe()

,name,hobby,age,edu,marital,class
count,132.000000,132.000000,132.000000,132.000000,132.000000,132.000000
mean,66.500000,2.000000,1.954545,1.954545,1.954545,1.840909
std,38.249183,0.819607,0.947988,0.947988,0.947988,0.769946
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,33.750000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,66.500000,2.000000,2.000000,2.000000,2.000000,2.000000
75%,99.250000,3.000000,2.000000,2.000000,2.000000,2.000000
max,132.000000,3.000000,4.000000,4.000000,4.000000,3.000000


# Train and Test Split

In [8]:
x = df.drop(['class'], axis=1)
y = df['class']

X_train,X_test,y_train,y_test=train_test_split(x, y, train_size=0.8, stratify = y, random_state=100)

In [9]:
y_train.shape, y_test.shape

((105,), (27,))

In [10]:
y_train.value_counts()/len(y_train)

2    0.390476
1    0.380952
3    0.228571
Name: class, dtype: float64

In [11]:
y_test.value_counts()/len(y_test)

1    0.407407
2    0.370370
3    0.222222
Name: class, dtype: float64

In [12]:
from sklearn.preprocessing import StandardScaler
Scaler_X = StandardScaler()
X_train = Scaler_X.fit_transform(X_train)
X_test = Scaler_X.transform(X_test)

# Model Building

In [13]:
model = list()
precision = list()
recall = list()
F1score = list()
AUCROC = list()
balanced_acc = list()
GMean = list()
sensi = list()
speci = list()

In [14]:
def test_eval(clf_model, X_test, y_test, algo=None):

    y_prob=clf_model.predict_proba(X_test)
    y_pred=clf_model.predict(X_test)

    print('Confusion Matrix')
    print('='*60)
    print(confusion_matrix(y_test,y_pred),"\n")
    print('Classification Report')
    print('='*60)
    print(classification_report(y_test,y_pred),"\n")
    print('='*60)
    print('AUC-ROC')
    print(roc_auc_score(y_test, y_prob, multi_class='ovr'), "\n")
    print('Balanced Accuracy')
    print(balanced_accuracy_score(y_test, y_pred),"\n")
    print('Geometric Mean Score')
    print(geometric_mean_score(y_test, y_pred, average='macro'),"\n")
    print('Sensitivity')
    print(sensitivity_score(y_test, y_pred, average='macro'),"\n")
    print('Specificity')
    print(specificity_score(y_test, y_pred, average='macro'))
          
    model.append(algo)
    precision.append(precision_score(y_test,y_pred, average='macro'))
    recall.append(recall_score(y_test,y_pred, average='macro'))
    F1score.append(f1_score(y_test,y_pred, average='macro'))
    AUCROC.append(roc_auc_score(y_test, y_prob, multi_class='ovr', average='macro'))
    balanced_acc.append(balanced_accuracy_score(y_test, y_pred))
    GMean.append(geometric_mean_score(y_test, y_pred, average='macro'))
    sensi.append(sensitivity_score(y_test, y_pred, average='macro'))
    speci.append(specificity_score(y_test, y_pred, average='macro'))


## Model-1: Logistic Regression

In [15]:
clf_LR = LogisticRegression(multi_class='ovr', solver='liblinear')
clf_LR.fit(X_train, y_train)

LogisticRegression(multi_class='ovr', solver='liblinear')

In [16]:
test_eval(clf_LR, X_test, y_test, 'Logistic Regression')

Confusion Matrix
[[8 3 0]
 [6 1 3]
 [0 3 3]] 

Classification Report
              precision    recall  f1-score   support

           1       0.57      0.73      0.64        11
           2       0.14      0.10      0.12        10
           3       0.50      0.50      0.50         6

    accuracy                           0.44        27
   macro avg       0.40      0.44      0.42        27
weighted avg       0.40      0.44      0.42        27
 

AUC-ROC
0.7062416885946298 

Balanced Accuracy
0.4424242424242424 

Geometric Mean Score
0.5603601343599942 

Sensitivity
0.4424242424242424 

Specificity
0.709733893557423


## Model-2: Decision Tree

In [17]:
clf_DT = DecisionTreeClassifier()
clf_DT.fit(X_train, y_train)

DecisionTreeClassifier()

In [18]:
test_eval(clf_DT, X_test, y_test, 'Decision Tree')

Confusion Matrix
[[10  1  0]
 [ 1  9  0]
 [ 0  0  6]] 

Classification Report
              precision    recall  f1-score   support

           1       0.91      0.91      0.91        11
           2       0.90      0.90      0.90        10
           3       1.00      1.00      1.00         6

    accuracy                           0.93        27
   macro avg       0.94      0.94      0.94        27
weighted avg       0.93      0.93      0.93        27
 

AUC-ROC
0.947961229946524 

Balanced Accuracy
0.9363636363636364 

Geometric Mean Score
0.9478902833687097 

Sensitivity
0.9363636363636364 

Specificity
0.9595588235294118


## Model-3: Gaussian NB

In [19]:
clf_NB = GaussianNB()
clf_NB.fit(X_train, y_train)

GaussianNB()

In [20]:
test_eval(clf_NB, X_test, y_test, 'Gaussian NB')

Confusion Matrix
[[10  1  0]
 [ 4  5  1]
 [ 0  0  6]] 

Classification Report
              precision    recall  f1-score   support

           1       0.71      0.91      0.80        11
           2       0.83      0.50      0.62        10
           3       0.86      1.00      0.92         6

    accuracy                           0.78        27
   macro avg       0.80      0.80      0.78        27
weighted avg       0.79      0.78      0.76        27
 

AUC-ROC
0.925864739835328 

Balanced Accuracy
0.8030303030303031 

Geometric Mean Score
0.8412008714619345 

Sensitivity
0.8030303030303031 

Specificity
0.8811858076563959


## Model-4: K-Nearest Neighbour

In [21]:
clf_KNN = KNeighborsClassifier()
clf_KNN.fit(X_train, y_train)

KNeighborsClassifier()

In [22]:
test_eval(clf_KNN, X_test, y_test, 'KNN')

Confusion Matrix
[[4 7 0]
 [4 6 0]
 [2 2 2]] 

Classification Report
              precision    recall  f1-score   support

           1       0.40      0.36      0.38        11
           2       0.40      0.60      0.48        10
           3       1.00      0.33      0.50         6

    accuracy                           0.44        27
   macro avg       0.60      0.43      0.45        27
weighted avg       0.53      0.44      0.44        27
 

AUC-ROC
0.738289760348584 

Balanced Accuracy
0.4323232323232323 

Geometric Mean Score
0.5495366167663114 

Sensitivity
0.4323232323232323 

Specificity
0.698529411764706


## MODEL-5 Support Vector Classifier

In [23]:
clf_SVC = SVC(probability=True)
clf_SVC.fit(X_train, y_train)

SVC(probability=True)

In [24]:
test_eval(clf_SVC, X_test, y_test, 'SVC')

Confusion Matrix
[[9 2 0]
 [0 9 1]
 [2 0 4]] 

Classification Report
              precision    recall  f1-score   support

           1       0.82      0.82      0.82        11
           2       0.82      0.90      0.86        10
           3       0.80      0.67      0.73         6

    accuracy                           0.81        27
   macro avg       0.81      0.79      0.80        27
weighted avg       0.81      0.81      0.81        27
 

AUC-ROC
0.9309389412330589 

Balanced Accuracy
0.794949494949495 

Geometric Mean Score
0.8473687882952738 

Sensitivity
0.794949494949495 

Specificity
0.9032446311858076


# MODEL-6 XGBoost

In [25]:
class BoostRFEWrap(BaseEstimator, BoostRFE):

    @if_delegate_has_method(delegate='estimator')
    def predict_proba(self, X):
        return self.predict(X, method='predict_proba')


xgb_params = {'max_depth': (5,10),
          'learning_rate': (0.001, 0.3),
          'n_estimators': (25, 200),
          'reg_alpha' : (0.1, 1),
          'reg_lambda': (0.1, 1),
          'subsample': (0.5,  0.9),
          'colsample_bytree': (0.5,1),
          'min_child_weight': (0,10),
          'gamma': (0,1)}

clf_XGB = BoostRFEWrap(XGBClassifier(), param_grid=xgb_params, importance_type='shap_importances', train_importance=False, min_features_to_select=1, step=1)

In [26]:
xclf = clf_XGB.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=6, verbose=0)


512 trials detected for ('max_depth', 'learning_rate', 'n_estimators', 'reg_alpha', 'reg_lambda', 'subsample', 'colsample_bytree', 'min_child_weight', 'gamma')

trial: 0001 ### iterations: 00010 ### eval_score: 0.25926
trial: 0002 ### iterations: 00015 ### eval_score: 0.18518
trial: 0003 ### iterations: 00003 ### eval_score: 0.40741
trial: 0004 ### iterations: 00005 ### eval_score: 0.40741
trial: 0005 ### iterations: 00001 ### eval_score: 0.11111
trial: 0006 ### iterations: 00008 ### eval_score: 0.07407
trial: 0007 ### iterations: 00003 ### eval_score: 0.33333
trial: 0008 ### iterations: 00003 ### eval_score: 0.33333
trial: 0009 ### iterations: 00003 ### eval_score: 0.33333
trial: 0010 ### iterations: 00003 ### eval_score: 0.22222
trial: 0011 ### iterations: 00002 ### eval_score: 0.14815
trial: 0012 ### iterations: 00002 ### eval_score: 0.18518
trial: 0013 ### iterations: 00008 ### eval_score: 0.03704
trial: 0014 ### iterations: 00006 ### eval_score: 0.11111
trial: 0015 ### iterations

trial: 0140 ### iterations: 00006 ### eval_score: 0.2963
trial: 0141 ### iterations: 00006 ### eval_score: 0.07407
trial: 0142 ### iterations: 00006 ### eval_score: 0.07407
trial: 0143 ### iterations: 00001 ### eval_score: 0.48148
trial: 0144 ### iterations: 00001 ### eval_score: 0.48148
trial: 0145 ### iterations: 00004 ### eval_score: 0.25926
trial: 0146 ### iterations: 00003 ### eval_score: 0.25926
trial: 0147 ### iterations: 00002 ### eval_score: 0.40741
trial: 0148 ### iterations: 00006 ### eval_score: 0.40741
trial: 0149 ### iterations: 00001 ### eval_score: 0.07407
trial: 0150 ### iterations: 00004 ### eval_score: 0.11111
trial: 0151 ### iterations: 00003 ### eval_score: 0.33333
trial: 0152 ### iterations: 00003 ### eval_score: 0.33333
trial: 0153 ### iterations: 00003 ### eval_score: 0.2963
trial: 0154 ### iterations: 00000 ### eval_score: 0.33333
trial: 0155 ### iterations: 00006 ### eval_score: 0.2963
trial: 0156 ### iterations: 00006 ### eval_score: 0.25926
trial: 0157 ### i

trial: 0282 ### iterations: 00012 ### eval_score: 0.25926
trial: 0283 ### iterations: 00002 ### eval_score: 0.14815
trial: 0284 ### iterations: 00004 ### eval_score: 0.25926
trial: 0285 ### iterations: 00006 ### eval_score: 0.03704
trial: 0286 ### iterations: 00005 ### eval_score: 0.11111
trial: 0287 ### iterations: 00003 ### eval_score: 0.2963
trial: 0288 ### iterations: 00007 ### eval_score: 0.40741
trial: 0289 ### iterations: 00003 ### eval_score: 0.11111
trial: 0290 ### iterations: 00003 ### eval_score: 0.14815
trial: 0291 ### iterations: 00006 ### eval_score: 0.40741
trial: 0292 ### iterations: 00003 ### eval_score: 0.40741
trial: 0293 ### iterations: 00001 ### eval_score: 0.07407
trial: 0294 ### iterations: 00001 ### eval_score: 0.11111
trial: 0295 ### iterations: 00003 ### eval_score: 0.33333
trial: 0296 ### iterations: 00003 ### eval_score: 0.33333
trial: 0297 ### iterations: 00004 ### eval_score: 0.22222
trial: 0298 ### iterations: 00011 ### eval_score: 0.14815
trial: 0299 ###

trial: 0424 ### iterations: 00003 ### eval_score: 0.33333
trial: 0425 ### iterations: 00004 ### eval_score: 0.22222
trial: 0426 ### iterations: 00000 ### eval_score: 0.25926
trial: 0427 ### iterations: 00002 ### eval_score: 0.18518
trial: 0428 ### iterations: 00006 ### eval_score: 0.25926
trial: 0429 ### iterations: 00004 ### eval_score: 0.14815
trial: 0430 ### iterations: 00003 ### eval_score: 0.14815
trial: 0431 ### iterations: 00001 ### eval_score: 0.48148
trial: 0432 ### iterations: 00001 ### eval_score: 0.48148
trial: 0433 ### iterations: 00001 ### eval_score: 0.2963
trial: 0434 ### iterations: 00003 ### eval_score: 0.22222
trial: 0435 ### iterations: 00003 ### eval_score: 0.40741
trial: 0436 ### iterations: 00005 ### eval_score: 0.40741
trial: 0437 ### iterations: 00001 ### eval_score: 0.11111
trial: 0438 ### iterations: 00002 ### eval_score: 0.14815
trial: 0439 ### iterations: 00003 ### eval_score: 0.33333
trial: 0440 ### iterations: 00003 ### eval_score: 0.33333
trial: 0441 ###

In [27]:
test_eval(xclf, X_test, y_test, 'XGB')

Confusion Matrix
[[10  1  0]
 [ 0 10  0]
 [ 0  0  6]] 

Classification Report
              precision    recall  f1-score   support

           1       1.00      0.91      0.95        11
           2       0.91      1.00      0.95        10
           3       1.00      1.00      1.00         6

    accuracy                           0.96        27
   macro avg       0.97      0.97      0.97        27
weighted avg       0.97      0.96      0.96        27
 

AUC-ROC
0.969229055258467 

Balanced Accuracy
0.9696969696969697 

Geometric Mean Score
0.9750298988361742 

Sensitivity
0.9696969696969697 

Specificity
0.9803921568627452


# MODEL-7 Stacking

In [28]:
clf_DT = DecisionTreeClassifier()
clf_SVC = SVC(probability=True)
clf_KNN = KNeighborsClassifier()
clf_LR = LogisticRegression()

sclf = StackingClassifier(classifiers=[clf_DT, clf_SVC, clf_KNN], use_probas=True, average_probas=False, meta_classifier=clf_LR)

In [29]:
sclf.fit(X_train, y_train)
test_eval(sclf, X_test, y_test, 'Stacking')

Confusion Matrix
[[9 2 0]
 [1 9 0]
 [0 0 6]] 

Classification Report
              precision    recall  f1-score   support

           1       0.90      0.82      0.86        11
           2       0.82      0.90      0.86        10
           3       1.00      1.00      1.00         6

    accuracy                           0.89        27
   macro avg       0.91      0.91      0.90        27
weighted avg       0.89      0.89      0.89        27
 

AUC-ROC
0.9172905525846703 

Balanced Accuracy
0.9060606060606061 

Geometric Mean Score
0.9228502343074843 

Sensitivity
0.9060606060606061 

Specificity
0.9399509803921569


# MODEL-8 Bagging

In [30]:
clf_bg = BaggingClassifier(clf_DT)

In [31]:
clf_bg.fit(X_train, y_train)
test_eval(clf_bg, X_test, y_test, 'Bagging')

Confusion Matrix
[[10  1  0]
 [ 2  8  0]
 [ 0  0  6]] 

Classification Report
              precision    recall  f1-score   support

           1       0.83      0.91      0.87        11
           2       0.89      0.80      0.84        10
           3       1.00      1.00      1.00         6

    accuracy                           0.89        27
   macro avg       0.91      0.90      0.90        27
weighted avg       0.89      0.89      0.89        27
 

AUC-ROC
0.9633801247771836 

Balanced Accuracy
0.903030303030303 

Geometric Mean Score
0.9207049276907529 

Sensitivity
0.903030303030303 

Specificity
0.9387254901960785


In [32]:
clf_eval_df = pd.DataFrame({'model':model,
                            'precision':precision,
                            'recall':recall,
                            'f1-score':F1score,
                            'AUC-ROC':AUCROC,
                            'balanced_acc':balanced_acc,
                            'GMean':GMean,
                            'sensitivity':sensi,
                            'specificity':speci})
clf_eval_df

,model,precision,recall,f1-score,AUC-ROC,balanced_acc,GMean,sensitivity,specificity
0,Logistic Regression,0.404762,0.442424,0.419216,0.706242,0.442424,0.560360,0.442424,0.709734
1,Decision Tree,0.936364,0.936364,0.936364,0.947961,0.936364,0.947890,0.936364,0.959559
2,Gaussian NB,0.801587,0.803030,0.782692,0.925865,0.803030,0.841201,0.803030,0.881186
3,KNN,0.600000,0.432323,0.453651,0.738290,0.432323,0.549537,0.432323,0.698529
4,SVC,0.812121,0.794949,0.800866,0.930939,0.794949,0.847369,0.794949,0.903245
5,XGB,0.969697,0.969697,0.968254,0.969229,0.969697,0.975030,0.969697,0.980392
6,Stacking,0.906061,0.906061,0.904762,0.917291,0.906061,0.922850,0.906061,0.939951
7,Bagging,0.907407,0.903030,0.903890,0.963380,0.903030,0.920705,0.903030,0.938725


In [33]:
clf_eval_df.to_excel("D:\Skripsi\Final\project_hayesroth_final.xlsx")